# Übungsaufgaben 8


## Aufgabe 1: xmlpos (Extraktion und Annotation von TEI-XML)
Schreiben Sie ein Python-Programm `xmlpos`, das aus einer Korpusdatei
den Dokumenteninhalt extrahiert, diesen dann satzweise POS-tagged
und eine XML-Datei folgender Form ausgibt:

```xml
<doc>
  <s id="s-0001">
    <w id="w-0001" pos="DET">Der</w>
	<w id="w-0002" pos="NOUN">Hase</w>
	...
  </s>
  <s id="s-0002">
  ...
</doc>
```

Anmerkungen:
* Verwenden Sie die [TEI](https://tei-c.org/)-kodierten Korpusdateien des [deutschen Textarchivs](https://www.deutschestextarchiv.de/), z.B:
  * Altman 1890: https://www.deutschestextarchiv.de/book/download_xml/altmann_elementarorganismen_1890
  * Brandes 1832: https://www.deutschestextarchiv.de/book/download_xml/brandes_naturlehre03_1832


> _XML-TEI (Text Encoding Initiative): weit verbreitetes XML-Dokumentenformat für die Kodierung von Textkorpora_    
  (Schema mit einheitlichen Elementnamen und Attributen für die editionswissenschaftliche und linguistische Annotation von Texten, u.a. über DTD definiert)

> _DTA (Deutsches Textarchiv): historisches deutsches Korpus_ (linguistisch annotiertes Volltextkorpus deutschsprachiger Texte aus der Zeit um 1600 bis 1900)
      

* Sie können die XML-Dateien auch mit Werkzeugen wie wget oder curl
  (man-Pages!)  herunterladen, oder auch Pythons `urllib` verwenden
* Sie können `nltk` zur Satzsegmentierung und zum POS-Tagging verwenden; alternativ
  dazu können Sie auch _stanza_ (oder _[spacy](https://spacy.io/)_) verwenden
* Stellen Sie sicher, dass der tatsächlichen Text aus den Dokumenten extrahiert wird;
  betrachten Sie hierzu folgendes (valides) XML-Dokument: `<a><b>1<c>2<d/>3</c></b>4<lb/>5</a>` 


```xml
 <a>
	<b>
		1
		<c>
			2
			<d/>
			3
		</c>
	</b>
	4
	<lb/>
	5
</a> 
``` 



#### *weiterführende Informationen:*

* [F-Strings](http://cissandbox.bentley.edu/sandbox/wp-content/uploads/2022-02-10-Documentation-on-f-strings-Updated.pdf)
* [Spacy](https://spacy.io/)
* [DTA](https://www.deutschestextarchiv.de/)
* [TEI](https://github.com/TEIC/TEI)

### Vorgehen:

1. XML-Dokument herunterladen (mit `requests`-Library oder `urllib`-Library)
2. Text (Inhalt von `<text>`-TEI-Element) mit `etree` aus Datei extrahieren:
   - mit XML-Parser über das XML-File iterieren (Namespace des TEI-Wurzelelements berücksichtigen!)
   -  Verwendung von Text-Extractor-Klasse (`TreeExtractor`; alternativ mit Funktionsaufruf: `gather_text`) und Hilfsfunktionen (`append_text`)
3. Satzsegmentierung(mit `NLTK.sent_tokenize()` und POS-Tagging (hier mit `spacy`)
4. Aufbau der neuen XML-Struktur mit `etree` (Satz-Segment- und Wort-POS-Informationen)

#### TEI-XML-Grundstruktur (Altmann-BSP):
```xml
<TEI xmlns="http://www.tei-c.org/ns/1.0">
    <teiHeader>
        ...
    </teiHeader>
    <text>
        <front>
            <titlePage type="halftitle">
                ...
            </titlePage>
        </front>
        <body>
            <div n="1">
                ...
                <p>
                    Seitdem von
                    <hi rendition="#k">Dujardin</hi>
                    die contraktile Substanz oder Sar¬
                    <lb/>
                    kode entdeckt war, hat dieselbe in Bezug auf die Deutung ihres
                    <lb/>
                    ...
                </p>
            </div>
        </body>
        <back>
            ...
        </back>
    </text>
</TEI>
```

---
### 1. append_text

- Hilfsfunktion zur Konkatenation von bereits gelesenem Text in der XML-Datei mit neu eingelesenem Text
- setzt getrennte Wörter am Zeilenende (`¬</lb>`) zusammen

In [7]:
def append_text(text, str):
    if not str or str == "":
        return text 
    
    str = str.strip()
    if text == "":
        return str 
    if text[-1] == '¬': # Trennerzeichen in XML-File
        return text[:-1] + str 
    return text + " " + str 

print(f'<<{append_text("abc", "   def ")}>>')
print(f'<<{append_text("unge¬", "legen")}>>')

<<abc def>>
<<ungelegen>>


---
### 2. Textextraktion mit Klasse `TextExtractor`

- Klasse zur Extraktion von Texts aus XML-Datei
- verwendet append_text in rekursiver Iteration über die Kinderknoten   
- erwartet root-ET-Objekt (z.B. `<text>`-Element im TEI-XML)

In [25]:
import xml.etree.ElementTree as ET 

class TextExtractor:
    def __init__(self, root):
        self.root = root 
        
    def extract(self):
        self.text = ""
        self.extract_rec(self.root) #Iterieren über Kinderknoten mit rekursiver Funktion
        return self.text
        
    def extract_rec(self, node):
        self.text = append_text(self.text, node.text)
        for c in node:  #Iterieren über Kinderknoten
            self.extract_rec(c) #rekursiver Aufruf (für Extraktion des Texts aus Kinder-Knoten)
        self.text = append_text(self.text, node.tail) #tail für Elemente vor dem schließenden Tag (im BSP: 4 und 5)
        
e = TextExtractor(ET.fromstring('<a><b>1<c>2<d/>3</c></b>4<lb/>5</a>'))
print(f'<<{e.extract()}>>')

<<1 2 3 4 5>>


#### Erläuterung: ohne append_text von node.tail (letzte Zeile in extract_rec) wird der Text vor dem schließenden Element nicht extrahiert:


In [24]:
import xml.etree.ElementTree as ET 

class TextExtractor:
    def __init__(self, root):
        self.root = root 
        
    def extract(self):
        self.text = ""
        self.extract_rec(self.root)
        return self.text
        
    def extract_rec(self, node):
        self.text = append_text(self.text, node.text)
        for c in node:
            self.extract_rec(c)
        #self.text = append_text(self.text, node.tail) #   <<<<<ohne tail: 3, 4 und 5 im BSP werden nicht extrahiert!

e = TextExtractor(ET.fromstring('<a><b>1<c>2<d/>3</c></b>4<lb/>5</a>'))
print(f'<<{e.extract()}>>')

<<1 2>>


### Alternativer `IterTextExtractor`

Verwendung von `itertext()` in Klasse

In [61]:
import xml.etree.ElementTree as ET 

class IterTextExtractor:
    def __init__(self, root):
        self.root = root 
        
    def iter_extract(self):
        self.text = ""
        for c in self.root.itertext():
            self.text = append_text(self.text, c)
        return self.text

e = IterTextExtractor(ET.fromstring('<a><b>1<c>2<d/>3</c></b>4<lb/>5</a>'))
print(f'<<{e.iter_extract()}>>')

<<1 2 3 4 5>>


### Alternative Textextraktionsfunktion: `gather_text()` 

Funktion statt Klasse (Verwendung siehe xmlpos_spacy.py)

In [23]:
def gather_text(node, text):
    text = append_text(text, node.text)
    for c in node: #Iterieren über Kinder-Knoten
        text = gather_text(c, text) #rekursiver Aufruf
    #return text
    return append_text(text, node.tail)

root = ET.fromstring('<a><b>1<c>2<d/>3</c></b>4<lb/>5</a>')
text = gather_text(root, "")
print(f'<<{text}>>')

<<1 2 3 4 5>>


---
### 3. Download der XML-Dateien und Verwendung der TextExtractor-Klasse

In [29]:
import xml.etree.ElementTree as ET 

import urllib.request
url = "https://www.deutschestextarchiv.de/book/download_xml/altmann_elementarorganismen_1890"
# url = "https://www.deutschestextarchiv.de/book/download_xml/brandes_naturlehre03_1832"


with urllib.request.urlopen(url) as f:
    root = ET.fromstring(f.read())

# TEI-Namespace (Namespace als Präfix der Elementnamen, z.B. http://www.tei-c.org/ns/1.0:element)
ns = {'tei': 'http://www.tei-c.org/ns/1.0'} #Namespace-Dictionary mit Abkürzungen

# Extraktion mit <tei:text> als Wurzelknoten (d.h. ohne Metadaten in <teiHeader>):
e = TextExtractor(root.find('tei:text', ns))
text = e.extract()
print(text[1002:1100])

zu sichern. Das Bewusstsein, dass uns hier die Grundprobleme der Biologie berühren, wird es hoffen


### 4. Download des [Spacy](https://spacy.io) Modells

In [ ]:
%%bash
#python -m spacy download de_core_news_sm

### 5. Importe

In [8]:
import sys
#!{sys.executable} -m pip install --upgrade pip spacy
import spacy
#!{sys.executable} -m pip install --upgrade pip nltk
from nltk import sent_tokenize
import xml.dom.minidom

### 6. POS-Tagging mit spaCy und Aufbau des XML-Baums

In [18]:
nlp = spacy.load('de_core_news_sm')
out = ET.Element('doc') #für Erzeugung XML-File
sid = 1 # sentence id
wid = 1 # word id

#Satzsegmentierung mit NLTK-Methode:
sents = sent_tokenize(text)

#POS-Tagging mit spaCy und Erzeugung XML-File:
for sent in sents:
    stag = ET.SubElement(out, 's') #Erzeugung von s-Element
    stag.attrib = {'id': f's-{sid:05d}'}
    sid += 1
    tokens = nlp(sent) #Analyse mit spaCy
    for token in tokens: #Erzeugen von w-Elementen (pro Satz)
        wtag = ET.SubElement(stag, 'w')
        wtag.text = token.text
        wtag.attrib = {'pos': token.pos_, 'id': f'w-{wid:05d}'}
        wid += 1

dom = xml.dom.minidom.parseString(ET.tostring(out))
print(dom.toprettyxml()[:1500])

<?xml version="1.0" ?>
<doc>
	<s id="s-00001">
		<w id="w-00001" pos="DET">DIE</w>
		<w id="w-00002" pos="NOUN">ELEMENTARORGANISMEN</w>
		<w id="w-00003" pos="PROPN">BEZIEHUNGEN</w>
		<w id="w-00004" pos="ADP">ZU</w>
		<w id="w-00005" pos="PROPN">DEN</w>
		<w id="w-00006" pos="PROPN">ZELLEN</w>
		<w id="w-00007" pos="ADP">VON</w>
		<w id="w-00008" pos="SPACE"> </w>
		<w id="w-00009" pos="PROPN">RICHARD</w>
		<w id="w-00010" pos="PROPN">ALTMANN</w>
		<w id="w-00011" pos="PROPN">MIT</w>
		<w id="w-00012" pos="PROPN">ZWEI</w>
		<w id="w-00013" pos="PROPN">ABBILDUNGEN</w>
		<w id="w-00014" pos="ADP">IM</w>
		<w id="w-00015" pos="PROPN">TEXT</w>
		<w id="w-00016" pos="CCONJ">UND</w>
		<w id="w-00017" pos="PROPN">XXI</w>
		<w id="w-00018" pos="PROPN">TAFELN</w>
		<w id="w-00019" pos="PUNCT">.</w>
	</s>
	<s id="s-00002">
		<w id="w-00020" pos="PROPN">LEIPZIG</w>
		<w id="w-00021" pos="PUNCT">,</w>
		<w id="w-00022" pos="PROPN">VERLAG</w>
		<w id="w-00023" pos="PROPN">VON</w>
		<w id="w-00024"

---

### Usage von xmlpos_spacy.py:

#### Einlesen aus XML-Datei, Tagging und Transformation mit xmlpos_spacy.py, Ausgabe in neue XML-Datei:

In [30]:
%%bash
cat altmann_elementarorganismen_1890.TEI-P5.xml | python3 xmlpos_spacy.py > altmann_elementarorganismen_1890_spacy.xml

#### Formatierte Ausgabe XML auf Kommandozeile mit xmllint:

In [98]:
%%bash
xmllint --format altmann_elementarorganismen_1890_spacy.xml | head -25

<?xml version="1.0"?>
<doc>
  <s id="s-1">
    <w pos="DET" id="w-1">DIE</w>
    <w pos="SPACE" id="w-2"> </w>
    <w pos="NOUN" id="w-3">ELEMENTARORGANISMEN</w>
    <w pos="CCONJ" id="w-4">UND</w>
    <w pos="PROPN" id="w-5">IHRE</w>
    <w pos="SPACE" id="w-6"> </w>
    <w pos="PROPN" id="w-7">BEZIEHUNGEN</w>
    <w pos="NOUN" id="w-8">ZU</w>
    <w pos="PROPN" id="w-9">DEN</w>
    <w pos="PROPN" id="w-10">ZELLEN</w>
    <w pos="PUNCT" id="w-11">.</w>
  </s>
  <s id="s-2">
    <w pos="ADP" id="w-12">VON</w>
    <w pos="SPACE" id="w-13">  </w>
    <w pos="X" id="w-14">RICHARD</w>
    <w pos="PROPN" id="w-15">ALTMANN</w>
    <w pos="PUNCT" id="w-16">.</w>
  </s>
  <s id="s-3">
    <w pos="ADP" id="w-17">MIT</w>
    <w pos="ADV" id="w-18">ZWEI</w>


In [101]:
%%bash
xmllint --format altmann_elementarorganismen_1890_spacy.xml | sed -n '106,150p'

  <s id="s-12">
    <w pos="NOUN" id="w-82">Vorbemerkung</w>
    <w pos="PUNCT" id="w-83">.</w>
  </s>
  <s id="s-13">
    <w pos="DET" id="w-84">Die</w>
    <w pos="ADJ" id="w-85">nachfolgenden</w>
    <w pos="NOUN" id="w-86">Capitel</w>
    <w pos="VERB" id="w-87">enthalten</w>
    <w pos="ADP" id="w-88">im</w>
    <w pos="NOUN" id="w-89">Wesentlichen</w>
    <w pos="DET" id="w-90">eine</w>
    <w pos="NOUN" id="w-91">theils</w>
    <w pos="VERB" id="w-92">erweiterte</w>
    <w pos="PUNCT" id="w-93">,</w>
    <w pos="ADV" id="w-94">theils</w>
    <w pos="ADJ" id="w-95">verk&#xFC;rzte</w>
    <w pos="NOUN" id="w-96">Zusammenstellung</w>
    <w pos="DET" id="w-97">derjenigen</w>
    <w pos="NOUN" id="w-98">Abhandlungen</w>
    <w pos="PUNCT" id="w-99">,</w>
    <w pos="PRON" id="w-100">welche</w>
    <w pos="ADV" id="w-101">bisher</w>
    <w pos="ADP" id="w-102">von</w>
    <w pos="PRON" id="w-103">mir</w>
    <w pos="ADP" id="w-104">&#xFC;ber</w>
    <w pos="DET" id="w-105">die</w>
  

#### Alternatives Herunterladen der XML-Datei mit curl:

In [26]:
%%bash
curl https://www.deutschestextarchiv.de/book/download_xml/altmann_elementarorganismen_1890 | python3 xmlpos_spacy.py > altmann_elementarorganismen_1890_spacy.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 22  385k   22 90112    0     0  49558      0  0:00:07  0:00:01  0:00:06 49539
100  385k  100  385k    0     0   202k      0  0:00:01  0:00:01 --:--:--  202k



## Aufgabe 2: Variante POS-Tagging mit stanza

- Abschließend wird hier noch eine Variante gezeigt, die stanza statt spaCy zum POS-Tagging verwendet
- auch zur Satzsegmentierung wird hier stanza verwendet


- MWT (Multi-Word Token Expansion) in Pipeline führt zu Abweichungen vom Originaltext, wenn man die über die words iteriert und diese ausgibt
  - da z.B. *zum* zu *zu* *dem* transformiert wird
- stattdessen muss über die tokens iteriert werden und POS-Infos aus den words eines token extrahiert werden
- vgl. https://stanfordnlp.github.io/stanza/mwt.html

#### xmlpos_stanza.py:

In [ ]:
import sys
import xml.etree.ElementTree as ET 
import stanza


def append_text(text, str):
    if not str or str == "":
        return text 
    
    str = str.strip()
    if text == "":
        return str 
    if text[-1] == '¬':
        return text[:-1] + str 
    return text + " " + str 


def gather_text(node, text):
    text = append_text(text, node.text)
    for c in  node:
        text = gather_text(c, text)
    return append_text(text, node.tail)
    

root = ET.fromstring(sys.stdin.read())    
out = ET.Element('doc')
nlp = stanza.Pipeline(lang='de', processors='tokenize, pos', download_method=None) #stanza-Pipeline
sid = 1 
tid = 1
text = gather_text(root, "")

for t in root.iter('{http://www.tei-c.org/ns/1.0}text'):
    text = gather_text(t, "")
    
    doc = nlp(text)  #POS-Annotation mit stanza
    sents = doc.sentences  #Tokenisierung: hier mit stanza statt NLTK.sent_tokenize
    for sent in sents: 
        stag = ET.SubElement(out, 's')
        stag.attrib = {'id': f"s-{sid}"}
        sid += 1
        
        tokens = sent.tokens #stanza-Methode .tokens (statt .words, wegen MWT)
        for token in tokens:
            ttag = ET.SubElement(stag, 'w')
            ttag.text = token.text
            word_upos = "+".join([word.upos for word in token.words]) #join der durch MWT von stanza getrennt analysierten word-POS-tags
            ttag.attrib = {'pos': word_upos, 'id': f"w-{tid}"}  #stanza
            tid += 1 
ET.dump(out)

#### Usage:

In [70]:
%%bash
cat altmann_elementarorganismen_1890.TEI-P5.xml | python3 xmlpos_stanza.py > altmann_elementarorganismen_1890_stanza.xml

2023-06-29 12:08:49 WARNING: Language de package default expects mwt, which has been added
2023-06-29 12:08:49 INFO: Loading these models for language: de (German):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |

2023-06-29 12:08:49 INFO: Using device: cpu
2023-06-29 12:08:49 INFO: Loading: tokenize
2023-06-29 12:08:49 INFO: Loading: mwt
2023-06-29 12:08:49 INFO: Loading: pos
2023-06-29 12:08:49 INFO: Done loading processors!


In [94]:
%%bash
xmllint --format altmann_elementarorganismen_1890_stanza.xml | sed -n '95,140p'

  <s id="s-10">
    <w pos="NOUN" id="w-75">Vorbemerkung</w>
    <w pos="PUNCT" id="w-76">.</w>
  </s>
  <s id="s-11">
    <w pos="DET" id="w-77">Die</w>
    <w pos="ADJ" id="w-78">nachfolgenden</w>
    <w pos="NOUN" id="w-79">Capitel</w>
    <w pos="VERB" id="w-80">enthalten</w>
    <w pos="ADP+DET" id="w-81">im</w>
    <w pos="NOUN" id="w-82">Wesentlichen</w>
    <w pos="DET" id="w-83">eine</w>
    <w pos="ADV" id="w-84">theils</w>
    <w pos="ADJ" id="w-85">erweiterte</w>
    <w pos="PUNCT" id="w-86">,</w>
    <w pos="ADV" id="w-87">theils</w>
    <w pos="ADJ" id="w-88">verk&#xFC;rzte</w>
    <w pos="NOUN" id="w-89">Zusammenstellung</w>
    <w pos="PRON" id="w-90">derjenigen</w>
    <w pos="NOUN" id="w-91">Abhandlungen</w>
    <w pos="PUNCT" id="w-92">,</w>
    <w pos="PRON" id="w-93">welche</w>
    <w pos="ADV" id="w-94">bisher</w>
    <w pos="ADP" id="w-95">von</w>
    <w pos="PRON" id="w-96">mir</w>
    <w pos="ADP" id="w-97">&#xFC;ber</w>
    <w pos="DET" id="w-98">die</w>
    <